In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# List of documents
s1 = "The Boy is Playing."
s2 = "The Girl is Singing."
s3 = "The Boy and Girl are enjoing."

df = pd.DataFrame()
df["documents"] = [s1,s2,s3]
df.head()

,documents
0,The Boy is Playing.
1,The Girl is Singing.
2,The Boy and Girl are enjoing.


In [4]:
# Preprocessing
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,The Boy is Playing.,the boy playing
1,The Girl is Singing.,the girl singing
2,The Boy and Girl are enjoing.,the boy and girl are enjoing


In [5]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc

In [7]:
df.head()

,documents,clean_documents
0,The Boy is Playing.,boy playing
1,The Girl is Singing.,girl singing
2,The Boy and Girl are enjoing.,boy girl enjoing


In [9]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()

array([[0.60534851, 0.        , 0.        , 0.79596054, 0.        ],
       [0.        , 0.        , 0.60534851, 0.        , 0.79596054],
       [0.51785612, 0.68091856, 0.51785612, 0.        , 0.        ]])

In [10]:
X.shape 
#A56   U(5,5). S()

(3, 5)

In [11]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)

In [12]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,boy playing,0.6006938726417858,-0.7071067811865468
1,girl singing,0.6006938726417836,0.7071067811865482
2,boy girl enjoing,0.8495094215244288,0.0000000000000011


In [13]:
# Features or words used as features 
dictionary = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
dictionary

['boy', 'enjoing', 'girl', 'playing', 'singing']

In [15]:
# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T

In [16]:
encoding_matrix

,topic_1,topic_2
boy,0.5567343500048622,-0.4280460350631179
enjoing,0.4007716354556667,0.0000000000000005
girl,0.5567343500048607,0.4280460350631193
playing,0.3312671303219817,-0.5628290964997662
singing,0.3312671303219801,0.5628290964997670
